### Join all files on county, state, and year

I joined crime, fema, acs, inflow and outflow data on county, state and year. Here's next steps: 1. sanity check to make sure that these look like they're joined properly. 2. check on inflow/outflow conflicting dates. 3. normalize data (crime rate per 100,000) 4. figure out strategy for imputing missing values for crime data and other data

In [ ]:
import pandas as pd

In [ ]:

ca_voter = pd.read_csv('/work/cleaned-csvs/ca_voter_reg_2014_2019.csv')
ca_weather = pd.read_csv('/work/cleaned-csvs/ca_weather.csv')
ca_economy = pd.read_csv('/work/cleaned-csvs/ca_economy.csv')
ca_school_expend = pd.read_csv('/work/cleaned-csvs/ca_school_expenditure.csv')
ca_crime = pd.read_csv('/work/cleaned-csvs/ca_crime.csv')

acs = pd.read_csv('/work/cleaned-csvs/acs.csv')
fema = pd.read_csv('/work/cleaned-csvs/fema.csv')
irs = pd.read_csv('/work/cleaned-csvs/irs_in_out')
position = pd.read_csv('/work/cleaned-csvs/position.csv')
mapper = pd.read_csv('/work/cleaned-csvs/us_counties.csv')

# Note: This file was created manually using data from https://en.wikipedia.org/wiki/Coastal_California
# It identifies if the county is on the coast 
ca_coastal = pd.read_csv('/work/assets/ca_coastal.csv',usecols=[0,1])

### Check For Missing Data

In [ ]:
#Check for missing data in inflow/outflow
unique_state_county_irs = len((irs['countyfips'].astype(str) + irs['statename']).value_counts())
print('irs years: ',irs['year'].unique())

print('Unique state/county values in irs: ',unique_state_county_irs)

num_years_irs = irs['year'].nunique()


print('Should have {} rows in irs'.format(num_years_irs * unique_state_county_irs))
print('Actual irs #rows: ',len(irs))
print('Missing {} rows in irs'.format(num_years_irs * unique_state_county_irs - len(irs)))


irs years:  [2014 2015 2016 2017 2018 2019]
Unique state/county values in irs:  3141
Should have 18846 rows in irs
Actual irs #rows:  18772
Missing 74 rows in irs


In [ ]:
#Check for missing ACS data
print('ACS num counties: ',acs['name'].nunique())
print('ACS years: ',acs['year'].unique())

ACS num counties:  3222
ACS years:  [2014 2015 2016 2017 2018 2019]


In [ ]:
#Check for missing CA data
ca_voter['county'] = ca_voter['county'].str.lower().str.strip()
print('CA voter num counties: ',ca_voter['county'].nunique())
print('CA voter years: ',ca_voter['year'].unique())
print('*************')
print('CA weather num counties: ',ca_weather['Location'].nunique())
print('CA weather years: ',ca_weather['year'].unique())
print('*************')
print('CA economy num counties: ',ca_economy['GeoFIPS'].nunique())
print('CA economy years: ',ca_economy['year'].unique())
print('*************')
print('CA school expenditure num counties: ',ca_school_expend['county_name'].nunique())
print('CA school expenditure years: ',ca_school_expend['year'].unique())
print('*************')
print('CA crime num counties: ',ca_crime['county_name'].nunique())
print('CA crime years: ',ca_crime['year'].unique())
print('*************')
print('CA coastal num counties: ',ca_coastal['County'].nunique())



CA voter num counties:  58
CA voter years:  [2014 2015 2016 2017 2018 2019]
*************
CA weather num counties:  58
CA weather years:  [2014 2015 2016 2017 2018 2019]
*************
CA economy num counties:  58
CA economy years:  [2014 2015 2016 2017 2018 2019]
*************
CA school expenditure num counties:  58
CA school expenditure years:  [2019 2018 2017 2016 2015 2014]
*************
CA crime num counties:  58
CA crime years:  [2014 2015 2016 2017 2018 2019]
*************
CA coastal num counties:  58


In [ ]:
#Check for missing Fema data
print('FEMA num counties: ',(fema['county'].astype(str) + fema['state'].astype(str)).nunique())
print('FEMA years: ',fema['year'].unique())


FEMA num counties:  2565
FEMA years:  [2016 2017 2018 2014 2019 2015]


In [ ]:
#Check for missing position data
print('Position num counties: ',(position['state'].astype(str) + position['county'].astype(str)).nunique())
#print('Position years: ',position['year'].unique())


Position num counties:  3126


### Join CA Weather, Voter, Economy, School Expenditure, Coastal , Crime and CA Codes Data

In [ ]:
ca_mapper = mapper[mapper['state_name']=='california']

#Prep Voter data
ca_voter['county'] = ca_voter['county'].str.lower().str.strip()
ca_voter.rename(columns={'county':'county_name'},inplace=True)

#Prep School Expenditure data
ca_school_expend['county_name'] = ca_school_expend['county_name'].str.lower().str.strip()

#Prep Weather data
ca_weather.rename(columns={'Location':'county_name'},inplace=True)
ca_weather['county_name'] = ca_weather['county_name'].str.lower()
ca_weather['county_name'] = ca_weather['county_name'].str.replace(' county', '')

#Prep Economy data
ca_economy['county'] = ca_economy['GeoFIPS'].astype(str).str[-3:].astype('int64')
ca_economy.drop(columns='GeoFIPS',inplace=True)

#Prep coastal data
ca_coastal['County'] = ca_coastal['County'].str.lower().str.strip()
ca_coastal.rename(columns={'County':'county_name'},inplace=True)

#Prep crime data 
ca_crime['county_name'] = ca_crime['county_name'].str.lower().str.strip()
ca_crime['county_name'] = ca_crime['county_name'].str.replace(' county', '')

#Merge CA data
ca_vote_weather = pd.merge(ca_voter, ca_weather, how='left',left_on=['county_name','year'],right_on=['county_name','year'])
ca_vote_weather_school = pd.merge(ca_vote_weather, ca_school_expend, how='left',on=['county_name','year'])

ca_vote_weather_mapper = pd.merge(ca_vote_weather_school, ca_mapper, how='left', left_on='county_name', right_on ='county_name')

ca_df = pd.merge(ca_vote_weather_mapper,ca_economy, how='left',on=['county','year']) \
            .merge(ca_crime, how = 'left',on=['county_name','year']) \
            .merge(ca_coastal, how = 'left', on = 'county_name')

ca_df
#Update crime fields to per capita using BEA number of people
ca_df['per_capita_num_violent_crimes'] = ca_df['num_violent_crimes']/ca_df['bea_number_of_people']
ca_df['per_capita_num_homicide'] = ca_df['num_homicide']/ca_df['bea_number_of_people']
ca_df['per_capita_num_rape'] = ca_df['num_rape']/ca_df['bea_number_of_people']
ca_df['per_capita_num_robbery'] = ca_df['num_robbery']/ca_df['bea_number_of_people']
ca_df['per_capita_num_agg_assault'] = ca_df['num_agg_assault']/ca_df['bea_number_of_people']
ca_df['per_capita_num_property_crimes'] = ca_df['num_property_crimes']/ca_df['bea_number_of_people']
ca_df['per_capita_num_burglary'] = ca_df['num_burglary']/ca_df['bea_number_of_people']
ca_df['per_capita_num_vehicle_theft'] = ca_df['num_vehicle_theft']/ca_df['bea_number_of_people']
ca_df['per_capita_num_larceny_theft'] = ca_df['num_larceny_theft']/ca_df['bea_number_of_people']
ca_df['per_capita_num_arson'] = ca_df['num_arson']/ca_df['bea_number_of_people']

#Drop fields no longer needed
ca_df = ca_df.drop(columns={'num_violent_crimes','num_homicide','num_rape','num_robbery',
                    'num_agg_assault','num_property_crimes','num_burglary','num_vehicle_theft',
                    'num_larceny_theft','num_arson'})
 

In [ ]:
ca_df.shape

(348, 39)

In [ ]:
ca_df.isna().sum()

county_name                           0
year                                  0
registered_pct                        0
democrat_pct                          0
republican_pct                        0
american_independent_pct              0
green_pct                             0
liberterian_pct                       0
peace_and_freedom_pct                 0
no_party_pct                          0
other_party_pct                       0
Location ID                           0
avg_temp                              0
min_temp                              0
max_temp                              0
total_precip_amt                      0
avg_daily_attendance_expense          0
state                                 0
county                                0
state_name                            0
bea_number_of_people                  0
per_capita_personal_income            0
per_capita_retirement_and_other       0
per_capita_unemploy_ins_comp          0
per_capita_curr_dollar_real_gdp       0


### Join ACS, Crime, FEMA, Inflow, and Outflow Data

In [ ]:
acs = acs.iloc[:,1:]
#acs.reset_index(inplace=True)
acs = acs[acs['state'] < 60] #remove puerto rico from data
acs.set_index(['county','state','year'],inplace = True)

fema = fema.iloc[:,1:]
fema.set_index(['county','state','year'],inplace = True)

col_dict = {'countyfips':'county','statefips':'state'}

irs = irs.iloc[:,1:]
irs = irs.drop(columns=['statename','countyname'])
irs.rename(columns=col_dict,inplace=True)
irs.set_index(['county','state','year'],inplace = True)




In [ ]:
#Check lengths
print('ACS #rows: ',len(acs))
print('Fema #rows: ',len(fema))
print('IRS #rows: ',len(irs))


#Make sure all datasets are unique on county, state, year
if len(acs.index.drop_duplicates()) != len(acs):
    print('Dupes in ACS')
if len(fema.index.drop_duplicates()) != len(fema):
    print('Dupes in FEMA')
if len(irs.index.drop_duplicates()) != len(irs):
    print('Dupes in IRS')


ACS #rows:  18852
Fema #rows:  5396
IRS #rows:  18772


In [ ]:
acs_fema = pd.merge(acs,fema,how = 'left',left_index = True, right_index = True)
print('acs_fema #rows: ',len(acs_fema))

fema_col = list(fema.columns) # all columns in fema dataset

acs_fema[fema_col] = acs_fema[fema_col].fillna(value = 0)

acs_fema.reset_index(inplace = True)


acs_fema #rows:  18852


In [ ]:

acs_fema.set_index(['county','state','year'],inplace=True)


In [ ]:
acs_fema.head()

,,,name,total_population,median_income,median_rent,median_home_value,educational_attainment,av_commute_time,perc_poverty,perc_white,perc_black,...,mud_landslide,severe_ice_storm,severe_storm,snow,tornado,toxic_substances,typhoon,volcano,county_name,state_name
county,state,year,,,,,,,,,,,,,,,,,,,,,
117,31,2014,"McPherson County, Nebraska",426.0,57763.0,657.0,132700.0,0.256173,28.689024,0.115023,0.981221,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
33,31,2014,"Cheyenne County, Nebraska",10044.0,54094.0,590.0,108400.0,0.238486,12.566143,0.133413,0.902031,0.002589,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
47,31,2014,"Dawson County, Nebraska",24205.0,48104.0,648.0,89700.0,0.151379,16.033507,0.136501,0.621277,0.040818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
71,31,2014,"Garfield County, Nebraska",1954.0,41776.0,334.0,84800.0,0.127719,10.801282,0.117195,0.981064,0.004606,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
125,31,2014,"Nance County, Nebraska",3667.0,46220.0,532.0,71000.0,0.144238,21.657526,0.111535,0.973275,0.000818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [ ]:
#Right now we are losing rows on this merge because we ae not synced up on the data
# Puerto Rico is the biggest difference between ACS and inflow/outflow data
# there are three counties in acs and not inflow outflow - two in Alaska and one in Hawaii

all_df = pd.merge(acs_fema,irs,how='inner',left_index=True,right_index=True)

In [ ]:
all_df.reset_index(inplace=True)

In [ ]:
all_df = pd.merge(all_df, position, how = 'left', left_on = ['county','state'], right_on=['county','state'])

In [ ]:
all_df.columns

Index(['county', 'state', 'year', 'name', 'total_population', 'median_income',
       'median_rent', 'median_home_value', 'educational_attainment',
       'av_commute_time', 'perc_poverty', 'perc_white', 'perc_black',
       'perc_american_indian', 'perc_asian', 'perc_hawaiian',
       'perc_other_race', 'perc_hispanic', 'perc_65_over',
       'perc_enrolled_undergrad', 'perc_unemployed', 'housing_per_capita',
       'perc_owner', 'perc_renter', 'perc_vacant', 'chemical', 'coastal_storm',
       'dam_levee_break', 'earthquake', 'fire', 'flood', 'hurricane',
       'mud_landslide', 'severe_ice_storm', 'severe_storm', 'snow', 'tornado',
       'toxic_substances', 'typhoon', 'volcano', 'county_name', 'state_name',
       'individual_inflow', 'individual_outflow', 'area_land', 'area_water',
       'longitude', 'latitude'],
      dtype='object')

In [ ]:
all_df.isna().sum()

county                     0
state                      0
year                       0
name                       0
total_population           0
median_income              0
median_rent                0
median_home_value          0
educational_attainment     0
av_commute_time            0
perc_poverty               0
perc_white                 0
perc_black                 0
perc_american_indian       0
perc_asian                 0
perc_hawaiian              0
perc_other_race            0
perc_hispanic              0
perc_65_over               0
perc_enrolled_undergrad    0
perc_unemployed            0
housing_per_capita         0
perc_owner                 0
perc_renter                0
perc_vacant                0
chemical                   0
coastal_storm              0
dam_levee_break            0
earthquake                 0
fire                       0
flood                      0
hurricane                  0
mud_landslide              0
severe_ice_storm           0
severe_storm  

In [ ]:
#There are two counties without long/lat data Shannon County and Wade
print(all_df[all_df['latitude'].isna()]['name'])


# See here for reason: https://www.economy.com/support/blog/buffet.aspx?did=50094FC4-C32C-4CCA-862A-264BC890E13B
# Shannon County, South Dakota (46-113)
# Changed name and code to Oglala Lakota County (46-102) effective May 1, 2015.

# Wade Hampton Census Area, Alaska (02-270)
# Changed name and code to Kusilvak Census Area (02-158) effective July 1, 2015.


390         Shannon County, South Dakota
1998    Wade Hampton Census Area, Alaska
Name: name, dtype: object


### Add National Data Features to CA Data

In [ ]:
#Pull out the California rows of national dataset to be joined with CA data
all_df_ca = all_df[all_df['state']==6]
all_df.head()

,county,state,year,name,total_population,median_income,median_rent,median_home_value,educational_attainment,av_commute_time,...,typhoon,volcano,county_name,state_name,individual_inflow,individual_outflow,area_land,area_water,longitude,latitude
0,117,31,2014,"McPherson County, Nebraska",426.0,57763.0,657.0,132700.0,0.256173,28.689024,...,0.0,0.0,0,0,0,0,2.224734e+09,2680669.0,-101.059936,41.567977
1,33,31,2014,"Cheyenne County, Nebraska",10044.0,54094.0,590.0,108400.0,0.238486,12.566143,...,0.0,0.0,0,0,617,505,3.098368e+09,292549.0,-102.994169,41.219994
2,47,31,2014,"Dawson County, Nebraska",24205.0,48104.0,648.0,89700.0,0.151379,16.033507,...,0.0,0.0,0,0,1205,1413,2.623887e+09,16431843.0,-99.820146,40.870282
3,71,31,2014,"Garfield County, Nebraska",1954.0,41776.0,334.0,84800.0,0.127719,10.801282,...,0.0,0.0,0,0,70,66,1.474558e+09,5321553.0,-98.991118,41.914290
4,125,31,2014,"Nance County, Nebraska",3667.0,46220.0,532.0,71000.0,0.144238,21.657526,...,0.0,0.0,0,0,169,117,1.143826e+09,17085959.0,-97.993273,41.397502


In [ ]:
ca_df.head()

,county_name,year,registered_pct,democrat_pct,republican_pct,american_independent_pct,green_pct,liberterian_pct,peace_and_freedom_pct,no_party_pct,...,per_capita_num_violent_crimes,per_capita_num_homicide,per_capita_num_rape,per_capita_num_robbery,per_capita_num_agg_assault,per_capita_num_property_crimes,per_capita_num_burglary,per_capita_num_vehicle_theft,per_capita_num_larceny_theft,per_capita_num_arson
0,alameda,2014,78.523182,55.470517,13.380565,2.041992,1.237333,0.517193,0.360684,21.249642,...,0.006442,0.000063,0.000263,0.003134,0.002981,0.035841,0.006116,0.007885,0.021839,0.000203
1,alpine,2014,87.015945,36.649215,29.842932,3.926702,1.701571,0.785340,0.130890,26.047120,...,0.003693,0.000000,0.000923,0.000000,0.002770,0.022161,0.009234,0.000000,0.012927,0.000000
2,amador,2014,78.208551,31.094336,44.162900,4.370613,0.572170,1.077027,0.235600,18.203673,...,0.002205,0.000027,0.000136,0.000272,0.001769,0.017123,0.005336,0.001089,0.010699,0.000109
3,butte,2014,70.024791,33.014476,36.155673,3.544590,1.230607,1.059547,0.427223,23.100687,...,0.003033,0.000049,0.000340,0.000667,0.001977,0.029666,0.007458,0.003400,0.018808,0.000389
4,calaveras,2014,76.305923,30.220186,41.909266,4.706665,0.886656,1.363233,0.365746,19.668982,...,0.002529,0.000000,0.000269,0.000201,0.002059,0.022137,0.006648,0.002082,0.013407,0.000045


In [ ]:
#Merge National features and CA features
ca_full_df = pd.merge(ca_df,all_df_ca,how='left',left_on = ['county','state','year'],right_on = ['county','state','year'])

In [ ]:
#Drop fields that are missing values and rename
ca_full_df = ca_full_df.drop(columns={'county_name_y','state_name_y'})
ca_full_df = ca_full_df.rename(columns={"county_name_x": "county_name", "state_name_x": "state_name"}) 


In [ ]:
ca_full_df

,county_name,year,registered_pct,democrat_pct,republican_pct,american_independent_pct,green_pct,liberterian_pct,peace_and_freedom_pct,no_party_pct,...,tornado,toxic_substances,typhoon,volcano,individual_inflow,individual_outflow,area_land,area_water,longitude,latitude
0,alameda,2014,78.523182,55.470517,13.380565,2.041992,1.237333,0.517193,0.360684,21.249642,...,0.0,0.0,0.0,0.0,72360,71551,1.909598e+09,216923745.0,-121.892901,37.646269
1,alpine,2014,87.015945,36.649215,29.842932,3.926702,1.701571,0.785340,0.130890,26.047120,...,0.0,0.0,0.0,0.0,0,0,1.912293e+09,12557304.0,-119.822348,38.596771
2,amador,2014,78.208551,31.094336,44.162900,4.370613,0.572170,1.077027,0.235600,18.203673,...,0.0,0.0,0.0,0.0,1605,1344,1.539934e+09,29470567.0,-120.653628,38.444795
3,butte,2014,70.024791,33.014476,36.155673,3.544590,1.230607,1.059547,0.427223,23.100687,...,0.0,0.0,0.0,0.0,7147,7017,4.238438e+09,105311003.0,-121.600520,39.667277
4,calaveras,2014,76.305923,30.220186,41.909266,4.706665,0.886656,1.363233,0.365746,19.668982,...,0.0,0.0,0.0,0.0,2292,2109,2.641829e+09,43797659.0,-120.552913,38.205371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,tulare,2019,63.594821,31.100510,39.187681,3.126937,0.278685,0.789410,0.256248,24.731500,...,0.0,0.0,0.0,0.0,10745,10970,1.249508e+10,37014644.0,-118.802020,36.220266
344,tuolumne,2019,77.943684,29.363418,41.220038,3.865746,0.553142,1.131285,0.275009,23.469483,...,0.0,0.0,0.0,0.0,2216,2091,5.752079e+09,138722707.0,-119.952509,38.026440
345,ventura,2019,81.371817,39.800167,29.640497,2.348195,0.402522,0.803259,0.263514,25.184192,...,0.0,0.0,0.0,0.0,27274,30430,4.771968e+09,947365005.0,-119.091061,34.444657
346,yolo,2019,74.955314,47.270375,19.551750,2.729625,0.712765,0.828919,0.358142,27.677267,...,0.0,0.0,0.0,0.0,9652,10789,2.628145e+09,22878452.0,-121.901632,38.683826


In [ ]:
ca_full_df.columns

Index(['county_name', 'year', 'registered_pct', 'democrat_pct',
       'republican_pct', 'american_independent_pct', 'green_pct',
       'liberterian_pct', 'peace_and_freedom_pct', 'no_party_pct',
       'other_party_pct', 'Location ID', 'avg_temp', 'min_temp', 'max_temp',
       'total_precip_amt', 'avg_daily_attendance_expense', 'state', 'county',
       'state_name', 'bea_number_of_people', 'per_capita_personal_income',
       'per_capita_retirement_and_other', 'per_capita_unemploy_ins_comp',
       'per_capita_curr_dollar_real_gdp', 'per_capita_num_jobs',
       'per_capita_farm_proprieter_jobs', 'per_capita_nonfarm_proprieter_jobs',
       'coastal_flag', 'per_capita_num_violent_crimes',
       'per_capita_num_homicide', 'per_capita_num_rape',
       'per_capita_num_robbery', 'per_capita_num_agg_assault',
       'per_capita_num_property_crimes', 'per_capita_num_burglary',
       'per_capita_num_vehicle_theft', 'per_capita_num_larceny_theft',
       'per_capita_num_arson', 'name', 

In [ ]:
#There are no nan values in ca_full_df
ca_full_df.isna().sum().sum()

0

### Write out final files

In [ ]:
#CA file
ca_full_df.to_csv('/work/cleaned-csvs/ca_counties_full_dataset.csv',index=False)

#National file
all_df.to_csv('/work/cleaned-csvs/national_counties_full_dataset.csv',index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6c76417-5fde-42f3-8920-755838dec3fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>